# Orc Automata - Evolución y Guerra

## Integrantes

- Carlos Fabián Morales Carrillo - 2240062
- Juan David Lipez Guevara - 2223102
- Yesmar Yesid Martínez Ortiz - 2241863
- Joseph Emanuel Sánchez Sierra - 2240959
- Jesús David Parada Palencia - 2240090
- David Alejandro Galvis Duarte - 2232522
- Andrés Felipe Prada Arciniegas - 2240069

## Nota **Importante**

Este colab es solo para mostrar el codigo, ya que usamos Pygame para ejecutar la simulacion y Colab no permite la ejecucion de la ventana grafica, para ejecutarlo siga las instrucciones del README del repositorio https://github.com/LipezJ/orc-life-game.

### Dependencias y configuraciones del proyecto
```toml
[build-system]
requires = ["setuptools>=68", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "orc-automata"
version = "0.1.0"
description = "Cellular automata of evolving orcs with a pygame renderer."
requires-python = ">=3.10"
authors = [{ name = "Student" }]
dependencies = [
    "pygame>=2.6",
]
readme = "README.md"

[project.scripts]
orc-automata = "orc_automata.main:main"

[tool.setuptools.packages.find]
where = ["src"]

[tool.setuptools.package-data]
orc_automata = ["assets/*.png"]
```

## Logica de la simulacion

### config.py

In [ ]:
from dataclasses import dataclass
from typing import Optional


@dataclass
class SimulationSettings:
    grid_width: int = 64
    grid_height: int = 40
    cell_size: int = 16
    tick_rate: int = 6
    classes: int = 3
    initial_orc_ratio: float = 0.08
    max_age: int = 220
    base_energy: float = 18.0
    energy_decay: float = 0.32
    move_cost: float = 0.55
    fight_reward: float = 2.6
    fight_cost: float = 0.8
    reproduction_threshold: float = 6.0
    reproduction_chance: float = 0.12
    reproduction_energy_share: float = 0.35
    reproduction_overpop_pop_threshold: int = 200
    reproduction_overpop_factor: float = 0.5
    mutation_rate: float = 0.15
    mutation_scale: float = 0.25
    humidity_penalty: float = 0.18
    humidity_bonus: float = 0.15
    forage_gain: float = 1.8
    forage_cost: float = 0.25
    rest_threshold: float = 5.5
    aggression_bias: float = 0.45
    herd_radius: int = 3
    herd_attraction: float = 0.55
    pair_seek_multiplier: float = 1.8
    escape_strength_threshold: float = 0.95
    escape_threat_radius: int = 2
    escape_threat_weight: float = 0.6
    biome_bonus: float = 0.16
    biome_penalty: float = 0.12
    peace_floor_count: int = 4
    skirmish_threshold: float = 0.9
    skirmish_cost_factor: float = 0.7
    group_support_bonus: float = 0.15
    group_support_radius: int = 2
    loner_grit_bonus: float = 0.35
    loner_grit_threshold: int = 3
    support_score_factor: float = 0.12
    virus_spawn_base: float = 0.0002
    virus_spawn_stressed: float = 0.0012
    virus_crowd_threshold: int = 6
    virus_crowd_pop_threshold: int = 150
    virus_crowd_multiplier: float = 5
    virus_spread_chance: float = 0.04
    virus_duration: int = 30
    virus_energy_penalty: float = 0.6
    virus_fight_penalty: float = 0.12
    max_population: int = 400
    overpop_base: float = 0.04
    overpop_scale: float = 0.18
    habitat_seek_radius: int = 4
    habitat_seek_bonus: float = 0.4
    habitat_bad_threshold: float = 0.48
    endangered_threshold: int = 15
    endangered_repro_bonus: float = 0.08
    endangered_repro_factor: float = 0.65
    kind_strength_mods: tuple[float, float, float] = (1.1, 0.9, 1.0)
    kind_agility_mods: tuple[float, float, float] = (0.95, 1.1, 1.0)
    kind_resilience_mods: tuple[float, float, float] = (1.0, 0.95, 1.1)
    seed: Optional[int] = None


DEFAULT_SETTINGS = SimulationSettings()


### enviroment.py

In [ ]:
from __future__ import annotations

import random
from dataclasses import dataclass
from typing import Iterable, Iterator, Optional, Tuple

from .orc import Orc


Coord = Tuple[int, int]


@dataclass
class Cell:
    x: int
    y: int
    orc: Optional[Orc] = None


class Environment:
    def __init__(self, width: int, height: int, rng: Optional[random.Random] = None) -> None:
        self.width = width
        self.height = height
        self.rng = rng or random.Random()
        self._grid: list[list[Optional[Orc]]] = [
            [None for _ in range(width)] for _ in range(height)
        ]
        self._humidity = self._generate_layer(bias=0.55, variation=0.22, vertical_pull=-0.2, smooth_passes=4)
        self._fertility = self._generate_layer(bias=0.5, variation=0.24, vertical_pull=0.16, smooth_passes=4)
        self._biome = self._generate_biomes()

    def in_bounds(self, x: int, y: int) -> bool:
        return 0 <= x < self.width and 0 <= y < self.height

    def get(self, x: int, y: int) -> Optional[Orc]:
        return self._grid[y][x]

    def place(self, orc: Orc, x: int, y: int) -> None:
        if not self.in_bounds(x, y):
            raise ValueError("Position out of bounds")
        if self._grid[y][x] is not None:
            raise ValueError("Cell already occupied")
        self._grid[y][x] = orc
        orc.position = (x, y)

    def move(self, orc: Orc, dest: Coord) -> None:
        dx, dy = dest
        if not self.in_bounds(dx, dy):
            return
        sx, sy = orc.position
        if self._grid[dy][dx] is None:
            self._grid[sy][sx] = None
            self._grid[dy][dx] = orc
            orc.position = (dx, dy)

    def remove(self, orc: Orc) -> None:
        x, y = orc.position
        if self._grid[y][x] is orc:
            self._grid[y][x] = None

    def neighbors(self, x: int, y: int) -> Iterator[Coord]:
        for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
            nx, ny = x + dx, y + dy
            if self.in_bounds(nx, ny):
                yield nx, ny

    def empty_neighbors(self, x: int, y: int) -> list[Coord]:
        return [coord for coord in self.neighbors(x, y) if self.get(*coord) is None]

    def occupied_neighbors(self, x: int, y: int) -> list[Coord]:
        return [coord for coord in self.neighbors(x, y) if self.get(*coord)]

    def iter_orcs(self) -> Iterable[Orc]:
        for row in self._grid:
            for maybe_orc in row:
                if maybe_orc:
                    yield maybe_orc

    def humidity_at(self, x: int, y: int) -> float:
        return self._humidity[y][x]

    def fertility_at(self, x: int, y: int) -> float:
        return self._fertility[y][x]

    def biome_at(self, x: int, y: int) -> int:
        return self._biome[y][x]

    def _generate_layer(
        self,
        bias: float,
        variation: float,
        vertical_pull: float,
        smooth_passes: int = 3,
    ) -> list[list[float]]:
        # Start with noisy values plus a vertical gradient and then smooth to get organic patches.
        layer: list[list[float]] = []
        denom = max(1, self.height - 1)
        for y in range(self.height):
            row: list[float] = []
            gradient = y / denom
            for _x in range(self.width):
                base = bias + vertical_pull * (gradient - 0.5)
                noise = self.rng.uniform(-variation, variation)
                row.append(base + noise)
            layer.append(row)
        for _ in range(max(0, smooth_passes)):
            layer = self._smooth(layer)
        return [[max(0.0, min(1.0, v)) for v in row] for row in layer]

    def _smooth(self, layer: list[list[float]]) -> list[list[float]]:
        smoothed: list[list[float]] = []
        for y in range(self.height):
            row: list[float] = []
            for x in range(self.width):
                cell_weight = 0.5
                count = 0
                neighbor_weight = 0.5
                for dy in (-1, 0, 1):
                    for dx in (-1, 0, 1):
                        if dx == 0 and dy == 0:
                            continue
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < self.width and 0 <= ny < self.height:
                            count += 1
                weight_per_neighbor = neighbor_weight / max(1, count)
                acc = layer[y][x] * cell_weight
                for dy in (-1, 0, 1):
                    for dx in (-1, 0, 1):
                        if dx == 0 and dy == 0:
                            continue
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < self.width and 0 <= ny < self.height:
                            acc += layer[ny][nx] * weight_per_neighbor
                row.append(acc if count else layer[y][x])
            smoothed.append(row)
        return smoothed

    def _generate_biomes(self) -> list[list[int]]:
        noise = self._generate_layer(bias=0.5, variation=0.35, vertical_pull=0.0, smooth_passes=3)
        # Flatten to compute thresholds for 3 roughly equal areas.
        flat = [v for row in noise for v in row]
        sorted_vals = sorted(flat)
        third = len(sorted_vals) // 3
        t1 = sorted_vals[third]
        t2 = sorted_vals[2 * third]
        biomes: list[list[int]] = []
        for row in noise:
            bio_row: list[int] = []
            for v in row:
                if v < t1:
                    bio_row.append(0)
                elif v < t2:
                    bio_row.append(1)
                else:
                    bio_row.append(2)
            biomes.append(bio_row)
        return biomes


### orc.py

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Tuple

Coord = Tuple[int, int]


@dataclass
class Orc:
    id: int
    position: Coord
    kind: int
    strength: float
    agility: float
    resilience: float
    energy: float
    age: int = 0
    infected: bool = False
    infection_timer: int = 0

    def tick(self, energy_decay: float) -> None:
        self.age += 1
        self.energy -= energy_decay

    def adjust_energy(self, delta: float) -> None:
        self.energy += delta

    def fitness(self) -> float:
        return (self.strength * 1.1) + (self.agility * 0.9) + (self.resilience * 0.8)

    def clone_with_mutation(
        self,
        rng,
        mutation_rate: float,
        mutation_scale: float,
        next_id: int,
    ) -> "Orc":
        def mutate(value: float) -> float:
            if rng.random() < mutation_rate:
                return max(0.1, value + rng.uniform(-mutation_scale, mutation_scale))
            return value

        return Orc(
            id=next_id,
            position=self.position,
            kind=self.kind,
            strength=mutate(self.strength),
            agility=mutate(self.agility),
            resilience=mutate(self.resilience),
            energy=self.energy * 0.6,
            age=0,
            infected=False,
            infection_timer=0,
        )

    def infect(self, duration: int) -> None:
        self.infected = True
        self.infection_timer = max(duration, 1)


### simulation.py

In [ ]:
from __future__ import annotations

import random
from dataclasses import dataclass
from typing import Dict, Iterable, Optional

from .config import DEFAULT_SETTINGS, SimulationSettings
from .environment import Environment
from .orc import Orc


@dataclass
class SimulationMetrics:
    tick: int
    population: int
    average_strength: float
    average_agility: float
    average_resilience: float


class Simulation:
    def __init__(
        self,
        settings: SimulationSettings = DEFAULT_SETTINGS,
        rng: Optional[random.Random] = None,
    ) -> None:
        self.settings = settings
        self.rng = rng or random.Random(settings.seed)
        self.environment = Environment(settings.grid_width, settings.grid_height, rng=self.rng)
        self.population: Dict[int, Orc] = {}
        self.tick = 0
        self._next_id = 1
        self._seed_initial_population()

    def _seed_initial_population(self) -> None:
        capacity = self.settings.grid_width * self.settings.grid_height
        target = int(capacity * self.settings.initial_orc_ratio)
        coords = [
            (x, y)
            for x in range(self.settings.grid_width)
            for y in range(self.settings.grid_height)
        ]
        self.rng.shuffle(coords)
        for x, y in coords[:target]:
            self._spawn_orc((x, y))

    def _spawn_orc(self, position: tuple[int, int]) -> Orc:
        biome = self.environment.biome_at(*position)
        kind = self._kind_for_biome(biome)
        orc = Orc(
            id=self._next_id,
            position=position,
            kind=kind,
            strength=self.rng.uniform(0.5, 1.5),
            agility=self.rng.uniform(0.5, 1.5),
            resilience=self.rng.uniform(0.5, 1.5),
            energy=self.settings.base_energy,
        )
        self._next_id += 1
        self._apply_kind_modifiers(orc)
        self.population[orc.id] = orc
        self.environment.place(orc, *position)
        return orc

    def reset(self) -> None:
        self.environment = Environment(
            self.settings.grid_width,
            self.settings.grid_height,
            rng=self.rng,
        )
        self.population.clear()
        self.tick = 0
        self._next_id = 1
        self._seed_initial_population()

    def step(self) -> None:
        order = list(self.population.values())
        self.rng.shuffle(order)
        for orc in order:
            if orc.id not in self.population:
                continue
            orc.tick(self.settings.energy_decay)
            self._apply_environment_pressure(orc)
            self._apply_social_context(orc)
            self._apply_disease(orc)
            if self._overpop_kill_check(orc):
                continue
            if self._is_dead(orc):
                self._remove_orc(orc)
                continue
            reproduced = self._maybe_reproduce(orc)
            if reproduced:
                continue
            self._take_action(orc)
        self.tick += 1

    def _apply_environment_pressure(self, orc: Orc) -> None:
        humidity = self.environment.humidity_at(*orc.position)
        if humidity < 0.45:
            penalty = (0.45 - humidity) * self.settings.humidity_penalty
            orc.adjust_energy(-penalty)
        elif humidity > 0.65:
            bonus = (humidity - 0.65) * self.settings.humidity_bonus
            orc.adjust_energy(bonus)
        biome = self.environment.biome_at(*orc.position)
        bonus, penalty = self._biome_effect(orc.kind, biome)
        if bonus:
            orc.adjust_energy(bonus)
        if penalty:
            orc.adjust_energy(-penalty)

    def _apply_social_context(self, orc: Orc) -> None:
        friends, foes = self._social_counts(orc)
        if friends >= 2:
            boost = min(3, friends) * self.settings.group_support_bonus
            orc.adjust_energy(boost)
        if foes >= friends + self.settings.loner_grit_threshold:
            orc.adjust_energy(self.settings.loner_grit_bonus)

    def _apply_disease(self, orc: Orc) -> None:
        if not orc.infected and self.rng.random() < self._virus_spawn_chance(orc):
            orc.infect(self.settings.virus_duration)
        if not orc.infected:
            return
        orc.infection_timer -= 1
        orc.adjust_energy(-self.settings.virus_energy_penalty)
        # Spread with low chance to adjacent occupied cells.
        for coord in self.environment.occupied_neighbors(*orc.position):
            target = self.environment.get(*coord)
            if target and not target.infected and self.rng.random() < self.settings.virus_spread_chance:
                target.infect(self.settings.virus_duration)
        if orc.infection_timer <= 0:
            orc.infected = False
            orc.infection_timer = 0

    def _overpop_kill_check(self, orc: Orc) -> bool:
        limit = self.settings.max_population
        if limit <= 0:
            return False
        current = len(self.population)
        if current <= limit:
            return False
        overload = (current - limit) / limit
        chance = min(0.75, self.settings.overpop_base + overload * self.settings.overpop_scale)
        if self.rng.random() < chance:
            self._remove_orc(orc)
            return True
        return False

    def _is_dead(self, orc: Orc) -> bool:
        return orc.energy <= 0 or orc.age > self.settings.max_age

    def _remove_orc(self, orc: Orc) -> None:
        self.environment.remove(orc)
        self.population.pop(orc.id, None)

    def _maybe_reproduce(self, orc: Orc) -> bool:
        pop_kind = self._kind_count(orc.kind)
        endangered = pop_kind <= self.settings.endangered_threshold
        threshold = self.settings.reproduction_threshold
        chance = self.settings.reproduction_chance
        if endangered:
            threshold *= self.settings.endangered_repro_factor
            chance = min(1.0, chance + self.settings.endangered_repro_bonus)
        # Si la poblacion total es alta, baja la probabilidad.
        if len(self.population) >= self.settings.reproduction_overpop_pop_threshold:
            chance *= self.settings.reproduction_overpop_factor

        if (orc.energy < threshold) or (self.rng.random() > chance):
            return False
        # Requiere al menos un aliado adyacente para reproducirse.
        ally_adjacent = any(
            (neighbor := self.environment.get(*coord)) and neighbor.kind == orc.kind
            for coord in self.environment.occupied_neighbors(*orc.position)
        )
        if not ally_adjacent:
            return False
        empties = self.environment.empty_neighbors(*orc.position)
        if not empties:
            return False
        dest = self.rng.choice(empties)
        energy_for_child = max(
            2.0,
            min(self.settings.base_energy, orc.energy * self.settings.reproduction_energy_share),
        )
        child = orc.clone_with_mutation(
            rng=self.rng,
            mutation_rate=self.settings.mutation_rate,
            mutation_scale=self.settings.mutation_scale,
            next_id=self._next_id,
        )
        self._next_id += 1
        child.energy = energy_for_child
        self.population[child.id] = child
        self.environment.place(child, *dest)
        orc.adjust_energy(-energy_for_child)
        return True

    def _take_action(self, orc: Orc) -> None:
        occupied = self.environment.occupied_neighbors(*orc.position)
        empties = self.environment.empty_neighbors(*orc.position)
        fertility_here = self.environment.fertility_at(*orc.position)
        low_energy = orc.energy < self.settings.rest_threshold

        if low_energy and self.rng.random() < 0.55:
            self._forage(orc, fertility_here)
            return

        target_coord = self._pick_target(orc, occupied)
        if target_coord:
            target = self.environment.get(*target_coord)
            if target and self._should_attack(orc, target):
                self._resolve_fight(orc, target)
                return

        if empties:
            dest = self._choose_move_target(orc, empties)
            self.environment.move(orc, dest)
            orc.adjust_energy(-self.settings.move_cost)
            if orc.energy < self.settings.rest_threshold and self.rng.random() < 0.35:
                fertility_moved = self.environment.fertility_at(*dest)
                self._forage(orc, fertility_moved)
            return

        if self.rng.random() < 0.4:
            self._forage(orc, fertility_here)

    def _resolve_fight(self, challenger: Orc, defender: Orc) -> None:
        if defender.id not in self.population or challenger.id not in self.population:
            return
        # Both lose a bit of energy just by engaging.
        challenger.adjust_energy(-self.settings.fight_cost * 0.5)
        defender.adjust_energy(-self.settings.fight_cost * 0.5)
        if challenger.energy <= 0:
            self._remove_orc(challenger)
            return
        if defender.energy <= 0:
            self._remove_orc(defender)
            return

        support_challenger = self._local_support_score(challenger)
        support_defender = self._local_support_score(defender)
        challenge_score = self._effective_fitness(challenger) + self.rng.uniform(-0.4, 0.4) + support_challenger
        defense_score = self._effective_fitness(defender) + self.rng.uniform(-0.4, 0.4) + support_defender
        diff = abs(challenge_score - defense_score)
        winner, loser = (challenger, defender) if challenge_score >= defense_score else (defender, challenger)

        if diff < self.settings.skirmish_threshold:
            # Skirmish: both retreat hurt, no death unless they were already exhausted.
            challenger.adjust_energy(-self.settings.fight_cost * self.settings.skirmish_cost_factor)
            defender.adjust_energy(-self.settings.fight_cost * self.settings.skirmish_cost_factor)
            winner.adjust_energy(self.settings.fight_reward * 0.4)
            if challenger.energy <= 0:
                self._remove_orc(challenger)
            if defender.energy <= 0:
                self._remove_orc(defender)
            return

        self._remove_orc(loser)
        winner.adjust_energy(self.settings.fight_reward - self.settings.fight_cost * 0.5)
        winner.strength += 0.05
        winner.resilience += 0.03
        if winner.energy <= 0:
            self._remove_orc(winner)

    def _forage(self, orc: Orc, fertility: float) -> None:
        gain = self.settings.forage_gain * (0.4 + fertility)
        gain *= self.rng.uniform(0.6, 1.2)
        orc.adjust_energy(gain - self.settings.forage_cost)

    def _pick_target(self, orc: Orc, occupied_neighbors) -> Optional[tuple[int, int]]:
        candidates: list[tuple[float, tuple[int, int]]] = []
        for coord in occupied_neighbors:
            target = self.environment.get(*coord)
            if not target:
                continue
            if target.kind == orc.kind:
                continue
            advantage = orc.fitness() - target.fitness()
            energy_gap = (orc.energy - target.energy) * 0.1
            score = advantage + energy_gap
            candidates.append((score, coord))
        if not candidates:
            return None
        candidates.sort(key=lambda item: item[0], reverse=True)
        best_score, best_coord = candidates[0]
        return best_coord if best_score > -0.3 else None

    def _choose_move_target(self, orc: Orc, empties: list[tuple[int, int]]) -> tuple[int, int]:
        current_score = self._env_score(orc.kind, orc.position)
        target_vec = self._seek_habitat_direction(orc, current_score)
        friends_adjacent = any(
            (neighbor := self.environment.get(*coord)) and neighbor.kind == orc.kind
            for coord in self.environment.occupied_neighbors(*orc.position)
        )
        low_pop = self._kind_count(orc.kind) <= self.settings.endangered_threshold
        low_strength = orc.strength <= self.settings.escape_strength_threshold
        weighted: list[tuple[float, tuple[int, int]]] = []
        for coord in empties:
            base = self._env_score(orc.kind, coord)
            herd_bonus = self._herd_bonus(orc, coord)
            if not friends_adjacent:
                herd_bonus *= self.settings.pair_seek_multiplier
            threat_penalty = 0.0
            if low_pop and low_strength:
                threat_penalty = self._threat_penalty(orc, coord)
            desirability = base + herd_bonus - threat_penalty
            if target_vec:
                vx, vy = target_vec
                dx, dy = coord[0] - orc.position[0], coord[1] - orc.position[1]
                dot = dx * vx + dy * vy
                norm = max(1e-3, (dx * dx + dy * dy) ** 0.5 * (vx * vx + vy * vy) ** 0.5)
                align = max(0.0, dot / norm)
                desirability += self.settings.habitat_seek_bonus * align
            if current_score < self.settings.habitat_bad_threshold:
                desirability += (base - current_score) * 0.8
            desirability += self.rng.uniform(-0.1, 0.1)
            weighted.append((desirability, coord))
        weighted.sort(key=lambda item: item[0], reverse=True)
        top = weighted[: min(3, len(weighted))]
        return self.rng.choice(top)[1]

    def _should_attack(self, challenger: Orc, defender: Orc) -> bool:
        advantage = challenger.fitness() - defender.fitness()
        if challenger.kind == defender.kind:
            return False
        if advantage < -0.6:
            return False
        # Avoid exterminating endangered populations.
        if self._kind_count(defender.kind) <= self.settings.endangered_threshold:
            return False
        # Protect small populations: if attacker group is small, avoid risky fights.
        if self._kind_count(challenger.kind) <= self.settings.peace_floor_count:
            return False
        aggression = self.settings.aggression_bias + advantage * 0.1
        energy_edge = challenger.energy - defender.energy
        if energy_edge > 2:
            aggression += 0.1
        return self.rng.random() < aggression

    def _herd_bonus(self, orc: Orc, coord: tuple[int, int]) -> float:
        radius = self.settings.herd_radius
        if radius <= 0:
            return 0.0
        ox, oy = coord
        total = 0
        for y in range(max(0, oy - radius), min(self.environment.height, oy + radius + 1)):
            for x in range(max(0, ox - radius), min(self.environment.width, ox + radius + 1)):
                neighbor = self.environment.get(x, y)
                if neighbor and neighbor.kind == orc.kind:
                    total += 1
        return total * (self.settings.herd_attraction / max(1, radius * radius))

    def _biome_effect(self, kind: int, biome: int) -> tuple[float, float]:
        # Simple cycle: biome 0 favorece clase 0, penaliza 1; biome 1 favorece 1, penaliza 2; biome 2 favorece 2, penaliza 0.
        if biome == kind:
            return (self.settings.biome_bonus, 0.0)
        if (biome + 1) % 3 == kind:
            return (0.0, 0.0)  # neutral
        return (0.0, self.settings.biome_penalty)

    def _biome_move_bonus(self, kind: int, biome: int) -> float:
        bonus, penalty = self._biome_effect(kind, biome)
        return bonus * 1.0 - penalty * 0.7

    def _virus_spawn_chance(self, orc: Orc) -> float:
        # Mayor probabilidad cuando esta en un bioma que lo penaliza.
        biome = self.environment.biome_at(*orc.position)
        _bonus, penalty = self._biome_effect(orc.kind, biome)
        base = self.settings.virus_spawn_stressed if penalty > 0 else self.settings.virus_spawn_base
        # Si hay mucha poblacion y esta rodeado de su clase, aumenta el riesgo.
        if len(self.population) >= self.settings.virus_crowd_pop_threshold:
            friends, _ = self._social_counts(orc)
            if friends >= self.settings.virus_crowd_threshold:
                base *= self.settings.virus_crowd_multiplier
        return base

    def _env_score(self, kind: int, coord: tuple[int, int]) -> float:
        humidity = self.environment.humidity_at(*coord)
        fertility = self.environment.fertility_at(*coord)
        biome = self.environment.biome_at(*coord)
        score = humidity * 0.35 + fertility * 0.5
        score += self._biome_move_bonus(kind, biome)
        return score

    def _seek_habitat_direction(self, orc: Orc, current_score: float) -> Optional[tuple[float, float]]:
        radius = self.settings.habitat_seek_radius
        if radius <= 0:
            return None
        best_score = current_score
        best_coord: Optional[tuple[int, int]] = None
        ox, oy = orc.position
        for y in range(max(0, oy - radius), min(self.environment.height, oy + radius + 1)):
            for x in range(max(0, ox - radius), min(self.environment.width, ox + radius + 1)):
                score = self._env_score(orc.kind, (x, y))
                if score > best_score + 0.05:
                    best_score = score
                    best_coord = (x, y)
        if best_coord is None:
            return None
        dx = best_coord[0] - ox
        dy = best_coord[1] - oy
        return (dx, dy)

    def _kind_count(self, kind: int) -> int:
        return sum(1 for o in self.population.values() if o.kind == kind)

    def _social_counts(self, orc: Orc) -> tuple[int, int]:
        radius = self.settings.group_support_radius
        x0, y0 = orc.position
        friends = 0
        foes = 0
        for y in range(max(0, y0 - radius), min(self.environment.height, y0 + radius + 1)):
            for x in range(max(0, x0 - radius), min(self.environment.width, x0 + radius + 1)):
                if x == x0 and y == y0:
                    continue
                other = self.environment.get(x, y)
                if not other:
                    continue
                if other.kind == orc.kind:
                    friends += 1
                else:
                    foes += 1
        return friends, foes

    def _local_support_score(self, orc: Orc) -> float:
        friends, foes = self._social_counts(orc)
        net = friends - foes * 0.6
        return net * self.settings.support_score_factor

    def _threat_penalty(self, orc: Orc, coord: tuple[int, int]) -> float:
        radius = self.settings.escape_threat_radius
        if radius <= 0:
            return 0.0
        ox, oy = coord
        enemies = 0
        for y in range(max(0, oy - radius), min(self.environment.height, oy + radius + 1)):
            for x in range(max(0, ox - radius), min(self.environment.width, ox + radius + 1)):
                if x == ox and y == oy:
                    continue
                other = self.environment.get(x, y)
                if other and other.kind != orc.kind:
                    enemies += 1
        return enemies * self.settings.escape_threat_weight

    def _effective_fitness(self, orc: Orc) -> float:
        fitness = orc.fitness()
        if orc.infected:
            fitness *= max(0.2, 1.0 - self.settings.virus_fight_penalty)
        return fitness

    def _apply_kind_modifiers(self, orc: Orc) -> None:
        idx = orc.kind % max(1, len(self.settings.kind_strength_mods))
        strength_mod = self.settings.kind_strength_mods[idx]
        agility_mod = self.settings.kind_agility_mods[idx]
        resilience_mod = self.settings.kind_resilience_mods[idx]
        orc.strength *= strength_mod
        orc.agility *= agility_mod
        orc.resilience *= resilience_mod

    def _kind_for_biome(self, biome: int) -> int:
        return biome % max(1, self.settings.classes)

    def metrics(self) -> SimulationMetrics:
        pop = list(self.population.values())
        count = len(pop)
        if count == 0:
            return SimulationMetrics(
                tick=self.tick,
                population=0,
                average_strength=0.0,
                average_agility=0.0,
                average_resilience=0.0,
            )
        return SimulationMetrics(
            tick=self.tick,
            population=count,
            average_strength=sum(o.strength for o in pop) / count,
            average_agility=sum(o.agility for o in pop) / count,
            average_resilience=sum(o.resilience for o in pop) / count,
        )

    def orcs(self) -> Iterable[Orc]:
        return self.population.values()

    def counts_by_kind(self) -> list[int]:
        counts = [0 for _ in range(max(1, self.settings.classes))]
        for orc in self.population.values():
            idx = orc.kind % len(counts)
            counts[idx] += 1
        return counts


### main.py


In [ ]:
from __future__ import annotations

import pygame

from .config import DEFAULT_SETTINGS, SimulationSettings
from .rendering.pygame_renderer import PygameRenderer
from .simulation import Simulation


def run_loop(settings: SimulationSettings) -> None:
    simulation = Simulation(settings=settings)
    renderer = PygameRenderer(settings=settings)
    clock = pygame.time.Clock()
    paused = False
    while renderer.running:
        actions = renderer.handle_events()
        if actions["quit"]:
            break
        if actions["toggle_pause"]:
            paused = not paused
        if actions["reset"]:
            simulation.reset()
        if not paused:
            simulation.step()
        renderer.draw(simulation, paused)
        clock.tick(settings.tick_rate)
    pygame.quit()


def main() -> None:
    run_loop(DEFAULT_SETTINGS)


if __name__ == "__main__":
    main()


## Ejecucion de la simulacion con **Pygame**

### rendering/colors.py

In [ ]:
from __future__ import annotations

from typing import Tuple

from ..orc import Orc


Color = Tuple[int, int, int]

DEEP_BG: Color = (14, 20, 28)
GRID: Color = (34, 46, 56)
HUD: Color = (230, 235, 238)
BIOME_TINTS: list[Color] = [
    (30, 10, 0),   # biome 0 slight reddish
    (0, 25, 5),    # biome 1 slight green
    (0, 5, 30),    # biome 2 slight blue
]
BIOME_BASE: list[Color] = [
    (132, 102, 74),  # tierra
    (90, 138, 82),   # pasto
    (46, 160, 164),  # agua/humedo
]


def _clamp(value: float, low: float, high: float) -> float:
    return max(low, min(value, high))


def _lerp(a: Color, b: Color, t: float) -> Color:
    return (
        int(a[0] + (b[0] - a[0]) * t),
        int(a[1] + (b[1] - a[1]) * t),
        int(a[2] + (b[2] - a[2]) * t),
    )


def color_for_orc(orc: Orc) -> Color:
    # Blend colors based on dominant traits to make clusters visible.
    trait_total = orc.strength + orc.agility + orc.resilience
    if trait_total <= 0:
        return (120, 120, 120)
    strength_ratio = _clamp(orc.strength / trait_total, 0.0, 1.0)
    agility_ratio = _clamp(orc.agility / trait_total, 0.0, 1.0)
    resilience_ratio = _clamp(orc.resilience / trait_total, 0.0, 1.0)
    base = (
        int(120 + 80 * strength_ratio),
        int(120 + 80 * agility_ratio),
        int(120 + 80 * resilience_ratio),
    )
    energy_factor = _clamp(orc.energy / 15.0, 0.2, 1.0)
    healthy = _lerp((30, 40, 40), base, energy_factor)
    if orc.infected:
        # Tint toward purple when infected.
        return _lerp(healthy, (160, 80, 200), 0.55)
    return healthy


def color_for_humidity(value: float) -> Color:
    level = _clamp(value, 0.0, 1.0)
    dry = (94, 73, 52)
    mid = (70, 100, 110)
    wet = (40, 125, 150)
    if level < 0.5:
        return _lerp(dry, mid, level * 2)
    return _lerp(mid, wet, (level - 0.5) * 2)


def blend_biome(base: Color, biome: int, strength: float = 0.18) -> Color:
    tint = BIOME_TINTS[biome % len(BIOME_TINTS)]
    return _lerp(base, tint, strength)


def color_for_cell(biome: int, humidity: float) -> Color:
    base = BIOME_BASE[biome % len(BIOME_BASE)]
    light = _clamp(0.6 + humidity * 0.25, 0.5, 0.95)
    return (
        int(base[0] * light),
        int(base[1] * light),
        int(base[2] * light),
    )


### pygame_renderer.py

In [ ]:
from __future__ import annotations

import random
from importlib import resources
from pathlib import Path
from typing import List, Optional

import pygame

from ..config import SimulationSettings
from ..simulation import Simulation
from .colors import DEEP_BG, GRID, HUD, blend_biome, color_for_cell, color_for_humidity, color_for_orc

INFECTION_GLOW = (170, 90, 210, 80)


class PygameRenderer:
    def __init__(self, settings: SimulationSettings) -> None:
        pygame.init()
        pygame.font.init()
        self.settings = settings
        self.cell_size = settings.cell_size
        self.width_px = settings.grid_width * self.cell_size
        self.height_px = settings.grid_height * self.cell_size
        self.screen = pygame.display.set_mode((self.width_px, self.height_px))
        pygame.display.set_caption("Orcos - Automata Celular")
        self.font = pygame.font.SysFont("consolas", 16)
        self.running = True
        self._noise_rng = random.Random(42)
        self.sprites = self._load_sprites()

    def handle_events(self) -> dict:
        events: dict = {"quit": False, "toggle_pause": False, "reset": False}
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                events["quit"] = True
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    events["quit"] = True
                if event.key == pygame.K_SPACE:
                    events["toggle_pause"] = True
                if event.key == pygame.K_r:
                    events["reset"] = True
        return events

    def draw(self, simulation: Simulation, paused: bool) -> None:
        self.screen.fill(DEEP_BG)
        self._draw_environment(simulation)
        self._draw_grid()
        self._draw_orcs(simulation)
        self._draw_hud(simulation, paused)
        pygame.display.flip()

    def _load_sprites(self) -> List[pygame.Surface]:
        names = ("orc1.png", "orc2.png", "orc3.png")
        sprites: list[pygame.Surface] = []
        for name in names:
            surface = self._load_resource_sprite(name)
            if surface is None:
                surface = self._load_fs_sprite(name)
            if surface is not None:
                sprites.append(surface)
        if not sprites:
            fallback = pygame.Surface((self.cell_size, self.cell_size), pygame.SRCALPHA)
            fallback.fill((200, 80, 80))
            sprites.append(fallback)
        if len(sprites) >= 2:
            sprites[1] = self._tint_sprite(sprites[1], (255, 240, 150))
        return sprites

    def _load_resource_sprite(self, filename: str) -> Optional[pygame.Surface]:
        try:
            data_ref = resources.files("orc_automata.assets") / filename
        except (FileNotFoundError, ModuleNotFoundError):
            return None
        if not data_ref.is_file():
            return None
        with resources.as_file(data_ref) as path:
            return self._scaled_sprite(Path(path))

    def _load_fs_sprite(self, filename: str) -> Optional[pygame.Surface]:
        root = Path(__file__).resolve().parents[3]
        path = root / "assets" / filename
        if not path.exists():
            return None
        return self._scaled_sprite(path)

    def _scaled_sprite(self, path: Path) -> pygame.Surface:
        sprite = pygame.image.load(str(path)).convert_alpha()
        # The provided files are sprite sheets with 5 frames in a row; take only the first frame.
        frames = 5
        frame_width = max(1, sprite.get_width() // frames)
        frame_rect = pygame.Rect(0, 0, frame_width, sprite.get_height())
        first_frame = sprite.subsurface(frame_rect).copy()
        size = max(1, int(self.cell_size * 1.3))
        scaled = pygame.transform.smoothscale(first_frame, (size, size))
        return scaled

    def _draw_environment(self, simulation: Simulation) -> None:
        for y in range(self.settings.grid_height):
            for x in range(self.settings.grid_width):
                humidity = simulation.environment.humidity_at(x, y)
                biome = simulation.environment.biome_at(x, y)
                color = blend_biome(color_for_cell(biome, humidity), biome, strength=0.08)
                rect = pygame.Rect(
                    x * self.cell_size,
                    y * self.cell_size,
                    self.cell_size,
                    self.cell_size,
                )
                self.screen.fill(color, rect)
                self._draw_cell_noise(x, y, biome)

    def _draw_grid(self) -> None:
        for x in range(0, self.width_px, self.cell_size):
            pygame.draw.line(self.screen, GRID, (x, 0), (x, self.height_px), 1)
        for y in range(0, self.height_px, self.cell_size):
            pygame.draw.line(self.screen, GRID, (0, y), (self.width_px, y), 1)

    def _draw_orcs(self, simulation: Simulation) -> None:
        for orc in simulation.orcs():
            x, y = orc.position
            cell_surface = pygame.Surface((self.cell_size, self.cell_size), pygame.SRCALPHA)
            if orc.infected:
                self._draw_infection_overlay(cell_surface)
            sprite = self._sprite_for_orc(orc)
            if sprite is not None:
                rect = sprite.get_rect()
                rect.center = (self.cell_size // 2, self.cell_size // 2)
                cell_surface.blit(sprite, rect)
            else:
                rect = pygame.Rect(0, 0, self.cell_size, self.cell_size)
                pygame.draw.rect(cell_surface, color_for_orc(orc), rect)
            self.screen.blit(cell_surface, (x * self.cell_size, y * self.cell_size))

    def _draw_hud(self, simulation: Simulation, paused: bool) -> None:
        metrics = simulation.metrics()
        counts = simulation.counts_by_kind()
        counts_text = " | ".join(f"C{idx}:{c}" for idx, c in enumerate(counts))
        lines = [
            f"Tick: {metrics.tick} {'(pausa)' if paused else ''}",
            f"Poblacion: {metrics.population}",
            f"Razas: {counts_text}",
            f"Fuerza promedio: {metrics.average_strength:.2f}",
            f"Agilidad promedio: {metrics.average_agility:.2f}",
            f"Resistencia promedio: {metrics.average_resilience:.2f}",
            "Teclas: Space pausa | R reinicia | Esc salir",
        ]
        padding = 6
        for idx, text in enumerate(lines):
            surface = self.font.render(text, True, HUD)
            self.screen.blit(surface, (padding, padding + idx * 18))

    def _sprite_for_orc(self, orc) -> Optional[pygame.Surface]:
        if not self.sprites:
            return None
        return self.sprites[orc.kind % len(self.sprites)]

    def _draw_cell_noise(self, x: int, y: int, biome: int) -> None:
        rng = random.Random((x * 73856093) ^ (y * 19349663) ^ (biome * 83492791))
        overlay = pygame.Surface((self.cell_size, self.cell_size), pygame.SRCALPHA)
        blotches = rng.randint(3, 6)
        for _ in range(blotches):
            r = max(2, self.cell_size // 6 + rng.randint(-1, 1))
            cx = rng.randint(r, self.cell_size - r)
            cy = rng.randint(r, self.cell_size - r)
            alpha = rng.randint(40, 70)
            # Subtle darker blotches per biome.
            blot_color = blend_biome(color_for_humidity(0.5), biome, strength=0.25)
            color = (blot_color[0], blot_color[1], blot_color[2], alpha)
            pygame.draw.circle(overlay, color, (cx, cy), r)
        self.screen.blit(overlay, (x * self.cell_size, y * self.cell_size))

    def _tint_sprite(self, sprite: pygame.Surface, rgb: tuple[int, int, int]) -> pygame.Surface:
        tinted = sprite.copy()
        tinted.fill(rgb + (0,), special_flags=pygame.BLEND_RGB_MULT)
        tinted.fill((15, 12, 0, 0), special_flags=pygame.BLEND_RGB_ADD)
        return tinted

    def _draw_infection_overlay(self, surface: pygame.Surface) -> None:
        radius = max(3, self.cell_size // 2)
        center = (self.cell_size // 2, self.cell_size // 2)
        halo = pygame.Surface(surface.get_size(), pygame.SRCALPHA)
        pygame.draw.circle(halo, INFECTION_GLOW, center, radius)
        # Halo se queda debajo porque luego se dibuja el sprite encima en la misma surface.
        surface.blit(halo, (0, 0))
